In [ ]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.2 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from sklearn.preprocessing import LabelEncoder
import json
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install huggingface_hub

^C


In [ ]:
# 1. Авторизация HF
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/huggingface_cli.py", line 52, in main
    service.run()
  File "/usr/local/lib/python3

In [ ]:
# Загрузить новый датасет из HF
dataset = load_dataset("takiholadi/kill-me-please-dataset")

README.md:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/36.5M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/3.18M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/27321 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2772 [00:00<?, ? examples/s]

In [ ]:
# Разделение на обучающую и тестовую выборки
train_dataset = dataset['train'].remove_columns(['votes', 'url', 'datetime'])
val_dataset = dataset['test'].remove_columns(['votes', 'url', 'datetime'])

In [ ]:
# Просмотр
train_dataset = pd.DataFrame(train_dataset)
print(train_dataset.head(10))

                                                text  \
0  Когда брился перепутал балончики с пеной и нам...   
1  Замок в туалете так и не починили. Новенькая э...   
2  У меня было три радости в жизни: моя работа, м...   
3  Задержался на пьянке по поводу праздника, по д...   
4  Вызвал сантехника прочистить раковину на кухне...   
5  Не ходил на учебу полгода, перед сессией решил...   
6  Пошла забирать ребенка из сада, по дороге встр...   
7  Мой парень сказал, что не может иметь детей и ...   
8  Милый сотрудник за неделю до праздника стал ин...   
9  Начальник сказал, что хочет дать мне премию, д...   

                              tags  
0                работа, внешность  
1                           работа  
2  работа, секс, друзья, отношения  
3                        отношения  
4                          техника  
5                            учеба  
6                           разное  
7                  секс, отношения  
8                        отношения  
9            

In [ ]:
# Проверка наличия None значений в столбце 'tags'
print("Количество None значений в train_dataset['tags']:", train_dataset['tags'].isna().sum())
val_dataset = pd.DataFrame(val_dataset)
print("Количество None значений в val_dataset['tags']:", val_dataset['tags'].isna().sum())

Количество None значений в train_dataset['tags']: 1
Количество None значений в val_dataset['tags']: 0


In [ ]:
train_dataset = train_dataset.dropna(subset=['tags'])

In [ ]:
# Перенумерация меток, чтобы они были последовательными
unique_labels = sorted(set(train_dataset['tags']) | set(val_dataset['tags']))
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
train_labels = [label_mapping[label] for label in train_dataset['tags']]
val_labels = [label_mapping[label] for label in val_dataset['tags']]
num_labels = len(unique_labels)

In [ ]:
# Предпросмотр новых меток
print("Новые метки после перекодирования:", label_mapping)

Новые метки после перекодирования: {'внешность': 0, 'деньги': 1, 'деньги, внешность': 2, 'деньги, отношения': 3, 'деньги, отношения, внешность': 4, 'деньги, отношения, техника': 5, 'деньги, техника': 6, 'деньги, техника, внешность': 7, 'друзья': 8, 'друзья, внешность': 9, 'друзья, деньги': 10, 'друзья, деньги, внешность': 11, 'друзья, деньги, отношения': 12, 'друзья, деньги, отношения, внешность': 13, 'друзья, деньги, отношения, техника': 14, 'друзья, деньги, техника': 15, 'друзья, здоровье': 16, 'друзья, здоровье, внешность': 17, 'друзья, здоровье, деньги': 18, 'друзья, здоровье, деньги, внешность': 19, 'друзья, здоровье, деньги, отношения': 20, 'друзья, здоровье, деньги, техника': 21, 'друзья, здоровье, отношения': 22, 'друзья, здоровье, отношения, техника': 23, 'друзья, здоровье, разное': 24, 'друзья, здоровье, разное, деньги': 25, 'друзья, здоровье, разное, отношения': 26, 'друзья, здоровье, родители': 27, 'друзья, здоровье, родители, внешность': 28, 'друзья, здоровье, родители, де

In [ ]:
# Сохранение словаря в файл для дальнейшего использования
with open('label_mapping.json', 'w') as f:
    json.dump(label_mapping, f)

In [ ]:
# 3. Загрузка токенизатора и модели
tokenizer = BertTokenizer.from_pretrained('cointegrated/rubert-tiny2')
model = BertForSequenceClassification.from_pretrained('cointegrated/rubert-tiny2', num_labels=num_labels)

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 4. Токенизация данных
train_encodings = tokenizer(train_dataset['text'].tolist(), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_dataset['text'].tolist(), truncation=True, padding=True, max_length=128)

In [ ]:
# 5. Создание PyTorch Dataset
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        # Конструктор класса, принимает токенизированные данные и метки
        self.encodings = encodings  # Сохранение токенизированных данных в объекте
        self.labels = labels  # Сохранение меток в объекте

    def __getitem__(self, idx):
        # Возвращает один элемент данных по индексу
        # Создает словарь, в котором ключи - это поля токенизированных данных, а значения - соответствующие тензоры
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])  # Добавление метки класса в элемент
        return item

    def __len__(self):
        # Возвращает количество элементов в датасете
        return len(self.labels)

In [ ]:
train_dataset = TextDataset(train_encodings, train_labels)  # Создание объекта TextDataset для обучающей выборки
val_dataset = TextDataset(val_encodings, val_labels)  # Создание объекта TextDataset для тестовой выборки

In [ ]:
# 6. Настройка параметров обучения
training_args = TrainingArguments(
    output_dir='./results',          # выходная директория
    num_train_epochs=3,              # количество эпох
    per_device_train_batch_size=16,  # размер batch для обучения
    per_device_eval_batch_size=16,   # размер batch для оценки
    evaluation_strategy='epoch',     # стратегия оценки
    save_strategy='epoch',           # стратегия сохранения модели
    logging_dir='./logs',            # директория для логов
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy'
)

In [ ]:
# 7. Определение метрик для оценки
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {'accuracy': acc, 'f1': f1}

In [ ]:
# 8. Создание объекта Trainer и запуск обучения
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,3.425000,3.910359,0.154762,0.089620
2,2.882800,3.629620,0.169913,0.104080
3,2.873700,3.565281,0.169913,0.115020


TrainOutput(global_step=5124, training_loss=3.312799903585239, metrics={'train_runtime': 182.3934, 'train_samples_per_second': 449.358, 'train_steps_per_second': 28.093, 'total_flos': 161697108418560.0, 'train_loss': 3.312799903585239, 'epoch': 3.0})

In [ ]:
# 9. Сохранение модели
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')

('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.txt',
 './fine_tuned_model/added_tokens.json')

In [ ]:
# 10. Проверка работы модели - генерация текстов и предсказание
sample_texts = [
    "Сегодня у меня был ужасный день на работе, все пошло не так.",
    "Пошел на прогулку с друзьями, чувствую себя счастливым.",
    "Сломался мой ноутбук, не знаю, что делать."
    "Сдал экзамен на отлично, очень рад своему успеху!",
    "Поссорился с коллегой на работе, чувствую себя подавленным.",
    "Записался на курсы по йоге, надеюсь улучшить свое здоровье и настроение."
]

In [ ]:
# Токенизация тестовых текстов
sample_encodings = tokenizer(sample_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')

In [ ]:
# Определение устройства (CPU или GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Перевод модели на устройство и в режим оценки
model.to(device)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-

In [ ]:
# Перемещение входных данных на то же устройство
sample_encodings = {key: val.to(device) for key, val in sample_encodings.items()}

In [ ]:
# Прогон текстов через модель и получение предсказаний
with torch.no_grad():
    outputs = model(**sample_encodings)
    predictions = outputs.logits.argmax(dim=-1)

In [ ]:
# Расшифровка предсказанных меток
with open('label_mapping.json', 'r') as f:
    label_mapping = json.load(f)
inv_label_mapping = {v: k for k, v in label_mapping.items()}
predicted_labels = [inv_label_mapping[int(pred)] for pred in predictions]

In [ ]:
# Вывод предсказаний
for text, label in zip(sample_texts, predicted_labels):
    print(f"Текст: {text}\nПредсказанные теги: {label}\n")

Текст: Сегодня у меня был ужасный день на работе, все пошло не так.
Предсказанные теги: работа

Текст: Пошел на прогулку с друзьями, чувствую себя счастливым.
Предсказанные теги: отношения

Текст: Сломался мой ноутбук, не знаю, что делать.Сдал экзамен на отлично, очень рад своему успеху!
Предсказанные теги: работа

Текст: Поссорился с коллегой на работе, чувствую себя подавленным.
Предсказанные теги: отношения

Текст: Записался на курсы по йоге, надеюсь улучшить свое здоровье и настроение.
Предсказанные теги: работа

